In [ ]:
%tensorflow_version 2.x

In [ ]:
!wget https://www.dropbox.com/sh/kpf9z73woodfssv/AAAw1_JIzpuVvwteJCma0xMla?dl=0

--2021-04-15 10:38:09--  https://www.dropbox.com/sh/kpf9z73woodfssv/AAAw1_JIzpuVvwteJCma0xMla?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.6.18, 2620:100:601c:18::a27d:612
Connecting to www.dropbox.com (www.dropbox.com)|162.125.6.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /sh/raw/kpf9z73woodfssv/AAAw1_JIzpuVvwteJCma0xMla [following]
--2021-04-15 10:38:09--  https://www.dropbox.com/sh/raw/kpf9z73woodfssv/AAAw1_JIzpuVvwteJCma0xMla
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucb1aee7df7335acf3518e43cf46.dl.dropboxusercontent.com/zip_download_get/AwDS4R_gbD2Svq3XCJy1EqAbqqAIXIei_iFgqRrGK5jirUWknA3t3JN0C5o0MIMwvbC_W-mP-POVQ3dYqFb90qspi188xaCAqw8FcriHrZ9J2Q [following]
--2021-04-15 10:38:10--  https://ucb1aee7df7335acf3518e43cf46.dl.dropboxusercontent.com/zip_download_get/AwDS4R_gbD2Svq3XCJy1EqAbqqAIXIei_iFgqRrGK5jirUWknA3t3JN0C5o0MIMwvbC_W-mP-POVQ3dYq

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from nltk import word_tokenize

In [ ]:
filter=pd.read_csv(r'/content/AAAw1_JIzpuVvwteJCma0xMla?dl=0',sep=',', encoding='latin-1')#,names=['ID','review','target'])

In [ ]:
filter

,PK,question_text,target
0,00002165364db923c7e6,How did Quebec nationalists see their province...,0.0
1,000032939017120e6e44,"Do you have an adopted dog, how would you enco...",0.0
2,0000412ca6e4628ce2cf,Why does velocity affect time? Does velocity a...,0.0
3,000042bf85aa498cd78e,How did Otto von Guericke used the Magdeburg h...,0.0
4,0000455dfa3e01eae3af,Can I convert montra helicon D to a mountain b...,0.0
...,...,...,...
1306119,ffffd48fb36b63db010c,Is foam insulation toxic?,0.0
1306120,ffffec519fa37cf60c78,How can one start a research project based on ...,0.0
1306121,ffffed09fedb5088744a,Who wins in a battle between a Wolverine and a...,0.0
1306122,PKÐÞ¾±´>g´>gPK,NaN,NaN


In [ ]:
filter.shape

(1306124, 3)

In [ ]:
filter.isnull().sum()

PK            0
question_text    2
target           2
dtype: int64

In [ ]:

for col in ['question_text','target']:
  print(col,',',filter[col].value_counts())

question_text , What is the difference between pack and bundle in hair extensions?                                               1
Why do I feel like vomiting if I study alot?                                                                     1
Could sociopathy technically be considered a form of PTSD with extreme emotional numbing as the main symptom?    1
How can I become a Raja?                                                                                         1
How was your experience at the first Quora meetup in Bhubaneshwar?                                               1
                                                                                                                ..
How many queens are there in a deck of cards?                                                                    1
Does it take more talent being a musician, or a filmmaker?                                                       1
Does estrogen affect the milk yield in dairy cows?              

In [ ]:
filter.drop([1306122,1306123],0,inplace=True)

In [ ]:
filter.isnull().sum()

PK            0
question_text    0
target           0
dtype: int64

In [ ]:

filter.shape

(1306122, 3)

In [ ]:
filter.drop(['PK'],1,inplace=True)

In [ ]:

filter.dtypes

question_text     object
target           float64
dtype: object

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np

from keras.models import Sequential
#from keras.layers.recurrent import LSTM,GRU
#from keras.layers.core import Dense, Activation, Dropout
from sklearn.metrics import roc_auc_score, accuracy_score
#from sklearn.utils import shuffle
from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping
from keras import models
from keras.models import load_model

from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.layers import LSTM, Activation, Dense, Input, Embedding, Dropout
from keras.models import Model
from nltk import word_tokenize

from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight

In [ ]:

!wget http://nlp.stanford.edu/data/glove.42B.300d.zip

--2021-04-15 10:38:24--  http://nlp.stanford.edu/data/glove.42B.300d.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.42B.300d.zip [following]
--2021-04-15 10:38:25--  https://nlp.stanford.edu/data/glove.42B.300d.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.42B.300d.zip [following]
--2021-04-15 10:38:25--  http://downloads.cs.stanford.edu/nlp/data/glove.42B.300d.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1877800501 (1.7G) [application/zip]
Sav

In [ ]:
!unzip /content/glove.42B.300d.zip

Archive:  /content/glove.42B.300d.zip
replace glove.42B.300d.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: [n]o
error:  invalid response [[n]o]
replace glove.42B.300d.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
!rm /content/glove.42B.300d.zip

In [ ]:

embeding_index={}

f=open('glove.42B.300d.txt',encoding='utf-8')

for line in f:
    values=line.split()
    word=values[0]
    coefs=np.asarray(values[1:],dtype='float32')
    embeding_index[word]=coefs
f.close()

In [ ]:
len(embeding_index['provide'])

300

In [ ]:
embeding_index['provide']

array([ 2.1902e-01, -6.6693e-02,  1.0538e-01, -2.0213e-01,  3.4797e-01,
       -8.4570e-03, -4.0350e+00,  3.4057e-01,  1.0484e-01,  2.1855e-01,
       -4.0837e-01,  2.7482e-01,  1.1953e-01,  7.4335e-02, -1.7069e-01,
       -4.0008e-01, -7.9120e-03, -3.9702e-01,  5.1159e-02, -2.3763e-02,
       -4.6144e-01,  4.7578e-02, -1.8110e-01,  3.9715e-03,  1.5718e-01,
       -1.8605e-01,  3.4680e-01,  1.7597e-01, -5.9390e-02, -3.0119e-01,
       -2.8326e-01, -2.6415e-01, -1.9358e-02,  6.7555e-03,  3.3796e-01,
       -3.5508e-01,  1.0336e-02, -5.0465e-01,  1.3932e-01, -2.7452e-01,
       -1.6377e-01,  5.0877e-01, -1.4303e-01, -2.0842e-01,  2.0463e-01,
        7.3660e-02, -1.8813e-01,  2.8356e-01, -2.7095e-01, -2.5136e-01,
       -1.2715e-01,  2.1164e-01, -4.4701e-01,  9.5515e-02,  4.5539e-01,
       -6.0036e-02,  1.4488e-02, -2.9433e-01, -5.5977e-01,  2.8814e-01,
        9.5750e-03,  2.2739e-01, -8.2802e-02,  3.0510e-01,  3.8501e-02,
        3.2902e-01, -1.1190e-02, -1.2909e-01,  3.6758e-01, -2.29

In [ ]:
len(embeding_index)

1917494

In [ ]:
X=filter['question_text']
Y=filter['target']

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(X,Y,test_size=0.2)
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((1044897,), (261225,), (1044897,), (261225,))

In [ ]:

class_weight=class_weight.compute_class_weight('balanced',np.unique(y_train),y_train)
print(class_weight)

[0.53291169 8.0960856 ]


In [ ]:

class_weight_dict = dict(enumerate(class_weight))
class_weight_dict

{0: 0.532911688083838, 1: 8.096085602268676}

In [ ]:

x_train

1283840    Pursuant to the Jurassic Park and Jurassic Wor...
164858     If I put away my pencil and never wrote again,...
419298                           What is your email address?
170722     Why are so many people against Israel when it ...
535948     What advice or inspiration can you tell to Mil...
                                 ...                        
732874     What are very small businesses, how do they work?
11374      Flow chart that will input unique values for A...
89045      So I like this girl, have confessed twice and ...
1102564                        How the number 189 is coming?
40802      What are some fun activities for teenagers in ...
Name: question_text, Length: 1044897, dtype: object

In [ ]:

from nltk import word_tokenize

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [ ]:
sent_lens=[len(word_tokenize(x)) for x in x_train]
print(max(sent_lens))

361


In [ ]:
np.percentile(sent_lens,99)

44.0

In [ ]:
max_len=44

tk=Tokenizer(char_level=False,split=' ')

tk.fit_on_texts(x_train)

seq_train=tk.texts_to_sequences(x_train)
seq_test=tk.texts_to_sequences(x_test)

vocab_size=len(tk.word_index)

seq_train_matrix=sequence.pad_sequences(seq_train,maxlen=max_len)
seq_test_matrix=sequence.pad_sequences(seq_test,maxlen=max_len)

In [ ]:
tk.word_index

{'the': 1,
 'what': 2,
 'is': 3,
 'a': 4,
 'to': 5,
 'in': 6,
 'of': 7,
 'i': 8,
 'how': 9,
 'and': 10,
 'do': 11,
 'are': 12,
 'for': 13,
 'you': 14,
 'can': 15,
 'why': 16,
 'it': 17,
 'my': 18,
 'that': 19,
 'if': 20,
 'with': 21,
 'on': 22,
 'or': 23,
 'have': 24,
 'be': 25,
 'does': 26,
 'from': 27,
 'your': 28,
 'an': 29,
 'which': 30,
 'should': 31,
 'when': 32,
 'get': 33,
 'best': 34,
 'would': 35,
 'as': 36,
 'people': 37,
 'some': 38,
 'there': 39,
 'who': 40,
 'will': 41,
 'like': 42,
 'at': 43,
 'not': 44,
 'about': 45,
 'by': 46,
 'they': 47,
 'was': 48,
 'did': 49,
 'any': 50,
 'we': 51,
 'so': 52,
 'good': 53,
 'me': 54,
 'their': 55,
 'one': 56,
 'india': 57,
 'has': 58,
 'after': 59,
 'most': 60,
 'where': 61,
 'make': 62,
 'this': 63,
 'but': 64,
 'more': 65,
 'all': 66,
 'think': 67,
 'many': 68,
 'between': 69,
 'time': 70,
 'than': 71,
 'much': 72,
 'other': 73,
 'life': 74,
 'someone': 75,
 'use': 76,
 'he': 77,
 'out': 78,
 'way': 79,
 'am': 80,
 'know': 81,
 'u

In [ ]:
vocab_size

195975

In [ ]:
seq_train_matrix.shape

(1044897, 44)

In [ ]:
seq_train

[[53495,
  5,
  1,
  12548,
  2027,
  10,
  12548,
  95,
  13968,
  41,
  37,
  88,
  137,
  19,
  618,
  5,
  20719,
  4807,
  13,
  1531,
  3,
  4,
  2756,
  436],
 [20, 8, 403, 431, 18, 5717, 10, 232, 3217, 494, 35, 173, 447],
 [2, 3, 28, 819, 1209],
 [16, 12, 52, 68, 37, 218, 711, 32, 17, 3, 202, 4, 3335, 1214],
 [2, 566, 23, 2757, 15, 14, 225, 5, 10620, 5230, 213],
 [2, 12, 38, 7, 1, 34, 4667, 7773, 13, 1432],
 [3, 16626, 611, 71, 1110, 3, 39, 4, 125, 69, 87, 23, 47, 12, 305, 160],
 [63,
  180,
  8,
  42,
  3,
  44,
  1619,
  1568,
  5,
  54,
  9,
  15,
  8,
  62,
  112,
  67,
  8,
  80,
  1619,
  1227],
 [2, 11, 14, 67, 7, 3131, 36, 4, 247, 5, 119, 4, 248, 167],
 [9, 11, 8, 93, 4, 13238, 181, 9262, 6, 57],
 [15, 679, 288, 142, 10916, 1099, 625, 2447, 7, 265, 1613],
 [2,
  11,
  14,
  98,
  12,
  7068,
  10,
  4391,
  19,
  476,
  31,
  86,
  22,
  4,
  621,
  1381,
  10,
  16],
 [2, 12, 1, 34, 750, 13, 3095, 1337],
 [2, 48, 28, 122, 6355, 21, 88200],
 [49, 14, 157, 336, 5251, 19,

In [ ]:
x_train.shape

(1044897,)

In [ ]:
len(seq_train),len(seq_test)

(1044897, 261225)

In [ ]:
seq_train_matrix.shape, seq_test_matrix.shape

((1044897, 44), (261225, 44))

In [ ]:
embedding_matrix=np.zeros((vocab_size+1,300))

for word,i in tk.word_index.items():
    embed_vector=embeding_index.get(word)
    if embed_vector is not None:
        embedding_matrix[i]=embed_vector

In [ ]:
len(embedding_matrix)

195976

In [ ]:
embedding_matrix.shape

In [ ]:
embedding_matrix

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [-0.20838   , -0.14932001, -0.017528  , ..., -0.54066002,
         0.21199   , -0.0094357 ],
       [ 0.13877   , -0.0021201 , -0.091039  , ...,  0.024614  ,
         0.46514001,  0.11363   ],
       ...,
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [-0.34219   ,  0.065755  ,  0.11131   , ...,  0.24890999,
        -0.72119999,  0.32857999],
       [ 0.1381    , -0.053236  , -0.097895  , ..., -0.083655  ,
        -0.38067001,  0.479     ]])

In [ ]:
inputs = Input(name='text_input',shape=[max_len])
embed=Embedding(vocab_size+1,300,input_length=max_len,mask_zero=True,
                weights=[embedding_matrix],trainable=False)(inputs)
lstm_layer1 = LSTM(512)(embed)
drop1 = Dropout(0.2)(lstm_layer1)
lstm_layer2 = Dense(256,activation='relu')(drop1)
drop2 = Dropout(0.2)(lstm_layer2)
lstm_layer3 = Dense(128,activation='relu')(drop2)
drop3 = Dropout(0.2)(lstm_layer3)
dense1 = Dense(25,activation='relu')(lstm_layer3)
drop4 = Dropout(0.2)(dense1)
final_layer = Dense(1,activation='sigmoid')(drop4)

model = Model(inputs=inputs,outputs=final_layer)

In [63]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
text_input (InputLayer)      [(None, 44)]              0         
_________________________________________________________________
embedding (Embedding)        (None, 44, 300)           58792800  
_________________________________________________________________
lstm (LSTM)                  (None, 512)               1665024   
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense (Dense)                (None, 256)               131328    
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               32896 

In [67]:
filepath='/content/weights-{epoch:02d}-{val_loss:.4f}.h5'
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, 
                             save_best_only=True)


In [69]:

earlystop = EarlyStopping(monitor='val_loss', patience=3,
                          verbose=1)

In [71]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])